In [ ]:
import pandas as pd
import numpy as np
import dill
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
# from sklearn.base import BaseEstimator, TransformerMixin

# Food Safety Model

This notebook details the development of a model to predict restaurant food safety from Google Maps reviews. The model is trained and validated using the [NYC Department of Health and Mental Hygiene restaurant inspection data](https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j).

## Inspection data cleanup

NYC DOHMH restaurant inspection data is catalogued in a large table with a lot of repeating information. To make life easier, I first reorganize the data into three tables:
- `restaurant_data`
    - Information about restaurants, including location and cuisine type
    - columns: `CAMIS`, `DBA`, `BORO`, `CUISINE DESCRIPTION`, `Latitude`, `Longitude`
- `review_codes`
    - Information pertaining to violation categories and violation severity (critical or non-critical)
    - columns: `VIOLATION CODE`, `CRITICAL FLAG`, `VIOLATION DESCRIPTION`
- `inspection_data`
    - Aggregated information from each individual restaurant inspection, including a list of violations and the restaurant grade
    - columns: `CAMIS`, `INSPECTION DATE`, `VIOLATION CODE`, `SCORE`, `GRADE`
    
Cleaned code is saved in files `data/restaurant_data.csv`, `data/violation_codes.csv`, and `data/inspection_data.csv`.

 ### Load data into a DataFrame

In [ ]:
DOHMH_df = pd.read_csv(
    'data/raw_data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv',
    # index_col = 'CAMIS',
    usecols = ['CAMIS', 'DBA', 'BORO', 'CUISINE DESCRIPTION',
                'INSPECTION DATE', 'VIOLATION CODE',
                'VIOLATION DESCRIPTION', 'CRITICAL FLAG',
                'SCORE', 'GRADE', 'Latitude', 'Longitude'],
    parse_dates = ['INSPECTION DATE'],
    dtype = {'CAMIS': 'str',
             'DBA': 'str',
             'BORO': 'str',
             'CUISINE DESCRIPTION': 'category',
             'SCORE': 'Int64',
             'GRADE': 'str'}
)

# DOHMH_df.info()

#### `restaurant_data`

In [ ]:
restaurant_data = DOHMH_df[[
    'CAMIS', 'DBA', 'BORO', 'CUISINE DESCRIPTION', 
    'Latitude', 'Longitude'
]].drop_duplicates()

restaurant_data.to_pickle('data/restaurant_data.pkl') #, index=False)
# restaurant_data.head(5)

#### `review_codes`

In [ ]:
# Review code key
violation_codes = DOHMH_df.loc[DOHMH_df['VIOLATION CODE'].notna(), 
    ['VIOLATION CODE', 'CRITICAL FLAG', 'VIOLATION DESCRIPTION']
].drop_duplicates(ignore_index = True)

violation_codes.to_pickle('data/violation_codes.pkl')#, index=False)
# violation_codes.head(5)

####  `inspection_data`

In [ ]:
# Inspection results - list of violation codes, score, 
# and grade for each CAMIS, INSPECTION DATE combo

def get_score(x):
    if x.min() == x.max():
        return x.min()
    if x.isna().all():
        return pd.NA
    
    return x.mode()


def get_grade(x):    
    g0 = x.iloc[0]
    for g in x.iloc[1:]:
        if g != g0 and not (pd.isna(g) and pd.isna(g0)):
            if pd.isna(g):
                continue
            elif pd.isna(g0):
                g0 = g
            else:
                raise ValueError(f'Grades disagree: {g} {g0}')
    return g0


inspection_data = (
    DOHMH_df[
        ['CAMIS', 'INSPECTION DATE', 'VIOLATION CODE', 'SCORE', 'GRADE']
    ]
    .groupby(['CAMIS', 'INSPECTION DATE'], as_index=False)
    .agg({
        'VIOLATION CODE' : lambda x: list(x),
        'SCORE' : lambda x: pd.Series.mode(x, dropna=False).max(),
        'GRADE': lambda x: pd.Series.mode(x),
    })
)

inspection_data.to_pickle('data/inspection_data.pkl')#, index=False)
# inspection_data.head(10)

## Google Maps review data cleanup

The data from Google Maps was downloaded in json format in several separate files. I want to reorganize this into two separate Pandas DataFrames covering all of the data:
- `maps_rest_data`
    - restaurant details
    - `CAMIS`, name, price level, rating, and categorical type
- `maps_review_data`
    - review details
    - `CAMIS`, author name, language, rating, text, time
    
The resulting data frames are saved in `data/maps_rest_data.csv` and `data/maps_review_data.csv`.

In [ ]:
import glob
maps_data_files = sorted(glob.glob('data/raw_data/gmaps_restaurant_data_*.pkd'))

In [ ]:
maps_rest_subframes = []
maps_review_subframes = []

for j, filename in enumerate(maps_data_files):
    f = open(filename, 'rb')
    maps_data = dill.load(f)
    
    # Get restaurant details, clean up column names
    maps_rest_subframes.append(
        pd.json_normalize(maps_data)[
            ['CAMIS', 'result.name', 'result.price_level', 'result.rating', 'result.types']
        ].rename(columns = {
            'result.name':'name',
            'result.price_level':'price_level',
            'result.rating':'rating',
            'result.types':'types'
        })
        .astype({'CAMIS':'str'})
    )

    # If restaurant has no reviews, create reviews field as an empty list
    # This avoids errors with pd.json_normalize
    for entry in maps_data:
        if not 'reviews' in entry['result']:
            entry['result']['reviews'] = []
    
    # Get review details, remove repetitive/unneeded columns
    maps_review_subframes.append(
        pd.json_normalize(
            maps_data, 
            ['result', 'reviews'], 'CAMIS'
        ).drop(columns=[
            'author_url', 
            'profile_photo_url', 
            'relative_time_description'])
    )

# Combine into single dataframe containing all restaurants
maps_rest_data = pd.concat(maps_rest_subframes)
maps_review_data = pd.concat(maps_review_subframes)

# Save
maps_rest_data.to_pickle('data/maps_rest_data.pkl')#, index=False)
maps_review_data.to_pickle('data/maps_review_data.pkl')#, index=False)

### Load data

Use this to load data that has already been cleaned/organized and saved to csv files.

In [ ]:
restaurant_data = pd.read_pickle('data/restaurant_data.pkl')
violation_codes = pd.read_pickle('data/violation_codes.pkl')
inspection_data = pd.read_pickle('data/inspection_data.pkl')

maps_rest_data = pd.read_pickle('data/maps_rest_data.pkl') #, dtype={'CAMIS':'str'})
maps_review_data = pd.read_pickle('data/maps_review_data.pkl')

In [ ]:
# maps_review_data.head(10)
inspection_data.info()

# A note on inspection data

Multiple inspections were performed at most restaurants. Data goes back to 2016. However, the limited reviews available from Google Maps (five maximum per restaurant) are mostly in 2021. See the below histograms.

In [ ]:
# ===============================================
# Plot histograms of inspection and review dates
# ===============================================

# Convert inspection date to datetime
inspection_data['INSPECTION DATE'] = pd.to_datetime(inspection_data['INSPECTION DATE'])

# Limit to latest inspections only
idx = inspection_data.groupby(['CAMIS'])['INSPECTION DATE'].transform(max) == inspection_data['INSPECTION DATE']
latest_inspections = inspection_data[idx]

# Convert Google Maps review dates to datetime
maps_review_data['time'] = pd.to_datetime(maps_review_data['time'], unit='s')


# Plot histograms of years of all inspections, latest inspections, and Google Maps reviews
plt.subplot(1,3,1)
sns.histplot(
    x = inspection_data.loc[inspection_data['INSPECTION DATE']>'1901','INSPECTION DATE'].dt.year,
    stat = 'count',
    binwidth = 1
) #.hist()
plt.xlim(2015,2022)
# plt.ylim(0,1)
plt.xticks([2015,2020])
plt.title('All inspections')

plt.subplot(1,3,2)
sns.histplot(
    x = latest_inspections.loc[latest_inspections['INSPECTION DATE']>'1901','INSPECTION DATE'].dt.year,
    stat = 'count',
    binwidth = 1
) #.hist()
plt.xlim(2015,2022)
# plt.ylim(0,1)
plt.xticks([2015,2020])
plt.title('Latest inspections')

plt.subplot(1,3,3)
sns.histplot(
    x = maps_review_data['time'].dt.year.to_list(),
    stat = 'count',
    binwidth = 1
) #.hist()
plt.xlim(2015,2022)
# plt.ylim(0,1)
plt.xticks([2015,2020])
plt.title('Maps reviews')

plt.tight_layout()

From here on, I assume a restaurant's latest inspection results are the best estimate of its current health code compliance as well as the best estimate of the restaurant's cleanliness when most of its available Google Maps reviews were written.

# Model development

## Predicting restaurant health inspection scores based on restaurant data (no review text)

First, I want to explore the utility of general restaurant data from Google Maps for predicting health inspection scores without the use of the review text itself. In this section, I examine correlations of `star_rating`, `price_level`, and `types` (establishment category) data with inspection scores.

First, join tables, do some type conversion, and compute the grade associated with inspection scores:

In [ ]:
# Join restaurant data with the inspection data
joined = maps_rest_data.merge(latest_inspections, on='CAMIS')

# Convert to float to avoid errors with seaborn plotting
joined = joined.astype({'SCORE':'float64'})

# Determine the associated NYC DOHMH grade for the restaurant's latest inspection score
joined['SCOREGRADE'] = pd.cut(
    joined['SCORE'],
    bins = [-1, 14, 28, 200],
    labels = ['A', 'B', 'C']
)

### By star rating

First, star rating. There is a slight negative correlation between star rating and inspection scores, meaning that restaurants with higher star ratings have better inspection results (low scores are good). This correlation is, however, very weak - star rating does not fully explain the variation in inspection results.

In [ ]:
# Linear model of correlation between Maps rating and inspection score
rating_model = Ridge()
rating_model.fit(joined[joined['rating'] > 3.0].dropna()[['rating']], joined[joined['rating'] > 3.0].dropna()['SCORE'])


# Plot prediction with the data
grouped = joined.groupby('SCOREGRADE')
for name, group in grouped:
    plt.scatter(group['rating'], group['SCORE'], alpha=0.05, label=f'{name} grade', s=20)

X = np.arange(1.0, 6.0, 1.0)
sns.lineplot(x=X, y=rating_model.predict(X.reshape(-1,1)), color='k', lw=2, label='prediction')

leg = plt.legend()
for lh in leg.legendHandles: 
    lh.set_alpha(1)

plt.xlabel('Google Maps rating')
plt.ylabel('Inspection Score')
plt.title(
    'Weak negative correlation between review rating and inspection score, coef = {:.2}, score = {:.3}'.format(
    rating_model.coef_[0],
    rating_model.score(joined.dropna()[['rating']], joined.dropna()['SCORE'])
    )
);

### By price level

Again, the correlation between price level and inspection scores is very weak. More expensive restaurants tend to achieve better health inspection results, however the variation in inspection results within each price range is greater than the variation between them.

In [ ]:
# Linear model of price level and inspection score correlation
pricelevel_model = Ridge()
pricelevel_model.fit(joined.dropna()[['price_level']], joined.dropna()['SCORE'])


# Plot prediction along with input data
sns.violinplot(x=joined['price_level'], y=joined['SCORE'], palette="Set2");

X = np.arange(1.0, 5.0, 1.0)

sns.lineplot(X-1, pricelevel_model.predict(X.reshape(-1,1)), color='k', lw=2, label='prediction')

plt.legend()
plt.title(
    'Weak positive correlation of inspection score with restaurant price level, coef = {:.2}, score = {:.3}'.format(
    pricelevel_model.coef_[0],
    pricelevel_model.score(joined.dropna()[['rating']], joined.dropna()['SCORE'])
    )
);

## By establishment category

Correlations between establishment categories and inspection results are stronger, with, e.g., supermarkets and bowling alleys performing relatively poorly and book stores and shopping malls performing relatively well.

In [ ]:
# First, need to convert the list of establishment categories to a string
# which will be fed to the CountVectorizer
def join_types(type_series):
    joinedtypes = []
    for l in type_series:
        try:
            joinedtypes.append(' '.join(l))
        except:
            joinedtypes.append([])
    
    return joinedtypes

joined['joinedtypes'] = join_types(joined['types'].to_list())
# joined.head(5)


# Build a linear model using CountVectorizer and Ridge regressor
category_model = Pipeline([
    ('count_types', CountVectorizer()),
    ('regressor', Ridge())
])

category_model.fit(joined.dropna()['joinedtypes'], joined.dropna()['SCORE'])

# Save dictionary of word tokens
typemap = {}
for key, value in category_model['count_types'].vocabulary_.items():
    typemap[value] = key

typecoefs = []
for j, coef in enumerate(category_model[-1].coef_):
    typecoefs.append((coef, typemap[j]))
    
typecoefs.sort()

# Plot correlation coefficients
fig=plt.figure(figsize=(6,12), dpi= 100)

sns.set_theme(style='whitegrid')
corr_coefs = [t[0] for t in typecoefs]
categories = [t[1] for t in typecoefs]
sns.set_color_codes('pastel')
ax = sns.barplot(x=corr_coefs, y = categories, color='b')
ax.tick_params(axis='y', labelsize=10)
plt.title('Correlation of establishment category with inspection scores, model score = {:.3}'
          .format(category_model.score(joined.dropna()['joinedtypes'], joined.dropna()['SCORE']))
         );

## Full restaurant data model

Now, we combine star rating, price level, and establishment category into a single model to predict inspection results. The model still leaves a lot of the variation in inspection results unexplained, with a final score of $R^2 = 0.00921$.

In [ ]:
# Full model based on all restaurant data

def join_types(type_series):
    joinedtypes = []
    for l in type_series:
        try:
            joinedtypes.append(' '.join(l))
        except:
            joinedtypes.append([])
    
    return joinedtypes

joined['joinedtypes'] = join_types(joined['types'].to_list())
joined.head(5)


encode_categories = ColumnTransformer([
    ('count_types', CountVectorizer(), 'joinedtypes'),
    ('pass', 'passthrough', ['price_level', 'rating']),
])


maps_rest_data_model = Pipeline([
    ('category_encode', encode_categories),
    ('regressor', Ridge())
])

maps_rest_data_model.fit(joined.dropna()[['price_level', 'rating', 'joinedtypes']], joined.dropna()['SCORE'])
print(
    'Full restaurant data model score: {:.3}'
    .format(
        maps_rest_data_model
        .score(
            joined.dropna()[['price_level', 'rating', 'joinedtypes']],
            joined.dropna()['SCORE']
        )
    )
)

# Predicting inspection results from Google Maps review text

To attempt to improve the ability of the model to predict health inspection scores, we can process the text of the reviews to find relations between specific words or phrases and inspection performance. This model performs similarly to the model above.

First, we combine all reviews for a particular restaurant into a single corpus of text.

In [ ]:
maps_review_text = (
    maps_review_data[['CAMIS','text']]
    .astype({'text':str, 'CAMIS':str})
    .groupby('CAMIS', as_index=False)
    .agg({
        'text': lambda x: ' '.join(x)
    })
)

text_model_df = maps_review_text.merge(latest_inspections[['CAMIS', 'SCORE']], on='CAMIS').dropna()
# text_model_df.head(10)

Next, we build the model pipeline. I'm using CountVectorizer to tokenize corpus words and bigrams, only allowing words consisting of letters, followed by a TfidfTransformer and the Ridge regressor.

In [ ]:
reviewtext_model = Pipeline([
    ('vectorizer', CountVectorizer(
        ngram_range=(1,2), 
        stop_words='english',
#         token_pattern=r'\b[a-zA-Z][a-zA-Z]+\b', # match only words (no numbers)
    )),
    ('tfidf_transform', TfidfTransformer()),
    ('regressor', Ridge()),
])

The feature space of the model is large compared to the object space, so we need to be careful about overfitting. Let's take a train/test split so we can fairly evaluate model performance.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    text_model_df['text'], 
    text_model_df['SCORE'], 
    test_size=0.2, 
    random_state=42
)

Now we can train the model and perform a grid search for cross validation and parameter selection.

In [ ]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(
    reviewtext_model, 
    param_grid = {
        'vectorizer__min_df':[10, 15], 
#         'vectorizer__max_df':[10000],#, 2000, 5000],
#         'vectorizer__stop_words':[STOP_WORDS],
        'regressor__alpha':[1.0, 10.0, 50.0],
    },
    n_jobs = -1,
)

gs.fit(X_train, y_train)

In [ ]:
print('Model score on training data: {:.3}'.format(gs.score(X_train, y_train)))
print('Model score on test data: {:.3}'.format(gs.score(X_test, y_test)))
print('\n')
print(f'Grid search best parameters: {gs.best_params_}')

View some example predictions:

In [ ]:
prediction = gs.predict(text_model_df['text'])

with_predict = text_model_df.copy()
with_predict['prediction'] = prediction

with_predict.head(10)

# Combined model

Now, I combine all of the above models into a single model to attempt to get the best possible performance I can acheive. This model is saved in order to be used in the Hugo web app.

In [ ]:
# ======================================
# SET UP THE MODEL PIPELINE
# ======================================
reviewtext_pipe = Pipeline([
    ('vectorizer', CountVectorizer(
        ngram_range=(1,2), 
        stop_words='english',
        min_df=5,
    )),
    ('tfidf_transform', TfidfTransformer()),
])
submodel_transform = ColumnTransformer([
    ('count_types', CountVectorizer(), 'joinedtypes'),
    ('pass', 'passthrough', ['price_level', 'rating']),
    ('reviewtext_data', reviewtext_pipe, 'text'),
])
full_model = Pipeline([
    ('submodels', submodel_transform),
    ('normalizer', Normalizer()),
    ('regressor', RidgeCV()), # Use RidgeCV for cross-validation
])

# =======================================
# PREPROCESSING
# =======================================
# Convert inspection date to datetime
inspection_data['INSPECTION DATE'] = pd.to_datetime(inspection_data['INSPECTION DATE'])

# Limit to latest inspections only
idx = inspection_data.groupby(['CAMIS'])['INSPECTION DATE'].transform(max) == inspection_data['INSPECTION DATE']
latest_inspections = inspection_data[idx]

# Restaurant data preprocessing
# Join restaurant data with the inspection data
joined = maps_rest_data.merge(latest_inspections, on='CAMIS')

# Convert to float to avoid errors with seaborn plotting
joined = joined.astype({'SCORE':'float64'})

# Category model preprocessing
def join_types(type_series):
    joinedtypes = []
    for l in type_series:
        try:
            joinedtypes.append(' '.join(l))
        except:
            joinedtypes.append([])    
    return joinedtypes
joined['joinedtypes'] = join_types(joined['types'].to_list())

# Review text preprocessing
maps_review_text = (
    maps_review_data[['CAMIS','text']]
    .astype({'text':str, 'CAMIS':str})
    .groupby('CAMIS', as_index=False)
    .agg({
        'text': lambda x: ' '.join(x)
    })
)
text_model_df = maps_review_text.merge(latest_inspections[['CAMIS']], on='CAMIS').dropna()

# Combine preprocessed data frames
full_model_df = joined.merge(text_model_df, on='CAMIS')
full_model_df[['rating', 'price_level', 'joinedtypes', 'text']].head(10)
full_model_df = full_model_df.dropna()

# =======================================
# FIT AND EVALUATE MODEL
# =======================================
X_train, X_test, y_train, y_test = train_test_split(
    full_model_df[['rating', 'price_level', 'joinedtypes', 'text']], 
    full_model_df['SCORE'], 
    test_size=0.2, 
    random_state=42
)

full_model.fit(X_train, y_train)
print('Training data score: {:.3}'.format(full_model.score(X_train, y_train)))
print('Testing data score: {:.3}'.format(full_model.score(X_test, y_test)))

In [ ]:
# =======================================
# SAVE MODEL
# =======================================
# from joblib import dump

# dump(full_model, 'Hugo_model.joblib')

# Other thoughts...

## Negative words

I want to find the words and phrases most uniquely associated with restaurants that perform poorly on health inspections so that I can highlight occurrences of those words or phrases in Google Maps reviews and display them in the web app. I limit the data to restaurants with inspection scores corresponding to A (0-13) or C (28+) grades and use a Naive Bayes classifier to find words in review text most associated with those grades.

This model is unsuccessful in the sense that the words identified as negative words are not indicative of any particular poor practices witnessed at restaurants - often they are simply food words or even a person's name.

In [ ]:
polar_inspections = text_model_df.copy()

polar_inspections['SCOREGRADE'] = pd.cut(
    polar_inspections['SCORE'],
    bins = [-1, 14, 28, 200],
    labels = ['A', 'B', 'C']
)

polar_inspections = polar_inspections[polar_inspections['SCOREGRADE'].isin(['A', 'C'])]

# polar_inspections.head(20)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

polar_model = Pipeline([
#     ('extractor', TextExtractor()),
    ('count_vectorizer', CountVectorizer(
        stop_words='english', 
        ngram_range=(1,2),
        min_df=6,
    )), #n_features=30000, 
    ('tfidf_transform', TfidfTransformer()),
    ('regressor', MultinomialNB()),
])

polar_model.fit(polar_inspections['text'], polar_inspections['SCOREGRADE'])

In [ ]:
# Save dictionary of word tokens
wordmap = {}
for key, value in polar_model['count_vectorizer'].vocabulary_.items():
    wordmap[value] = key

# Get polarity of words
polarity = polar_model[-1].feature_log_prob_

# Get polar words
negativity = polarity[1,:] - polarity[0,:]
negative_words = [wordmap[i] for i in np.argsort(negativity)]
print(negative_words[-100:])
print(len(negative_words))

At first glance, the most negatively polar words don't appear particularly negative or associated with restaurant cleanliness in any way, but maybe we can still use them to help make predictions of a restaurant's inspection score.

In [ ]:
polar_inspections['negative_count'] = polar_inspections['text'].str.count('|'.join(negative_words[-100:]))

sns.violinplot(x=polar_inspections['negative_count'], y=polar_inspections['SCORE'], palette="Set3");

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    polar_inspections[['negative_count']], 
    polar_inspections['SCORE'], 
    test_size=0.2, 
    random_state=12
)

In [ ]:
negative_words_model = Ridge()
negative_words_model.fit(X_train, y_train)

print('Training data score: {:.3}'.format(negative_words_model.score(X_train, y_train)))
print('Testing data score: {:.3}'.format(negative_words_model.score(X_test, y_test)))

# Old Code

This code (most of which was early data exploration) is no longer central to the notebook. I may decide to bring some portion of it back later, so I'm keeping it here for convenience.

In [ ]:
# # Let's get a first look at the variations in review ratings and inspection scores with restaurant price level
# plt.subplot(1,2,1)
# sns.violinplot(x=joined['price_level'], y=joined['rating'], palette="Set2");
# plt.subplot(1,2,2)
# sns.violinplot(x=joined['price_level'], y=joined['SCORE'], palette="Set2");
# plt.tight_layout()

In [ ]:
# sns.histplot(
#     data=joined.loc[joined['GRADE'].isin(['A', 'B', 'C']), ['rating', 'price_level','GRADE']].sort_values('GRADE'),
#     x='GRADE',
#     hue='price_level',
#     multiple='stack',
#     palette='Set2',
# #     stat='density'
# )

In [ ]:
# plt.subplot(1,3,1)
# sns.histplot(
#     data=joined.loc[joined['GRADE']=='A', ['rating', 'price_level','GRADE']].sort_values('GRADE'),
#     x='GRADE',
#     hue='price_level',
#     multiple='stack',
#     palette='Set2',
#     stat='percent'
# )
# plt.subplot(1,3,2)
# sns.histplot(
#     data=joined.loc[joined['GRADE']=='B', ['rating', 'price_level','GRADE']].sort_values('GRADE'),
#     x='GRADE',
#     hue='price_level',
#     multiple='stack',
#     palette='Set2',
#     stat='percent'
# )
# plt.subplot(1,3,3)
# sns.histplot(
#     data=joined.loc[joined['GRADE']=='C', ['rating', 'price_level','GRADE']].sort_values('GRADE'),
#     x='GRADE',
#     hue='price_level',
#     multiple='stack',
#     palette='Set2',
#     stat='percent'
# )
# plt.tight_layout()

In [ ]:
# joined_stars = joined
# joined_stars['stars'] = joined['rating'].round(0)

# axs = []
# axs.append(plt.subplot(1,4,1))
# sns.histplot(
#     data=(
#         joined
#         .loc[(joined['price_level']==1) & joined['GRADE'].isin(['A','B','C']), ['stars', 'price_level','GRADE']]
#         .sort_values('GRADE'))
#     ,
#     x='price_level',
#     hue='GRADE',
#     multiple='stack',
#     palette='Set2',
#     stat='percent'
# )
# plt.title('Price: \$')

# axs.append(plt.subplot(1,4,2))
# sns.histplot(
#     data=(
#         joined
#         .loc[(joined['price_level']==2) & joined['GRADE'].isin(['A','B','C']), ['stars', 'price_level','GRADE']]
#         .sort_values('GRADE'))
#     ,
#     x='price_level',
#     hue='GRADE',
#     multiple='stack',
#     palette='Set2',
#     stat='percent'
# )
# plt.title('\$\$')

# axs.append(plt.subplot(1,4,3))
# sns.histplot(
#     data=(
#         joined
#         .loc[(joined['price_level']==3) & joined['GRADE'].isin(['A','B','C']), ['stars', 'price_level','GRADE']]
#         .sort_values('GRADE'))
#     ,
#     x='price_level',
#     hue='GRADE',
#     multiple='stack',
#     palette='Set2',
#     stat='percent'
# )
# plt.title('\$\$\$')

# axs.append(plt.subplot(1,4,4))
# sns.histplot(
#     data=(
#         joined
#         .loc[(joined['price_level']==4) & joined['GRADE'].isin(['A','B','C']), ['stars', 'price_level','GRADE']]
#         .sort_values('GRADE'))
#     ,
#     x='price_level',
#     hue='GRADE',
#     multiple='stack',
#     palette='Set2',
#     stat='percent'
# )
# plt.title('\$\$\$\$')

# for ax in axs[1:]:
#     ax.set(ylabel=None)
#     ax.set(yticks=[])
#     ax.get_legend().remove()
# for ax in axs:
#     ax.set(xlabel=None)
#     ax.set(xticks=[])
    
# plt.tight_layout()

In [ ]:
# sns.histplot(
#     data=(
#         joined
#         .loc[joined['GRADE'].isin(['A','B','C']), ['stars', 'price_level','GRADE']]
#         .sort_values('GRADE'))
#     ,
#     x='price_level',
#     palette='Set2',
#     stat='count'
# )

In [ ]:
# joined_stars = joined
# joined_stars['stars'] = joined['rating'].round(0)

# axs = []
# axs.append(plt.subplot(1,5,1))
# sns.histplot(
#     data=(
#         joined
#         .loc[(joined['stars']==1) & joined['GRADE'].isin(['A','B','C']), ['stars', 'price_level','GRADE']]
#         .sort_values('GRADE'))
#     ,
#     x='stars',
#     hue='GRADE',
#     multiple='stack',
#     palette='Set2',
#     stat='percent'
# )
# plt.title('One star')

# axs.append(plt.subplot(1,5,2))
# sns.histplot(
#     data=(
#         joined
#         .loc[(joined['stars']==2) & joined['GRADE'].isin(['A','B','C']), ['stars', 'price_level','GRADE']]
#         .sort_values('GRADE'))
#     ,
#     x='stars',
#     hue='GRADE',
#     multiple='stack',
#     palette='Set2',
#     stat='percent'
# )
# plt.title('Two stars')

# axs.append(plt.subplot(1,5,3))
# sns.histplot(
#     data=(
#         joined
#         .loc[(joined['stars']==3) & joined['GRADE'].isin(['A','B','C']), ['stars', 'price_level','GRADE']]
#         .sort_values('GRADE'))
#     ,
#     x='stars',
#     hue='GRADE',
#     multiple='stack',
#     palette='Set2',
#     stat='percent',
# )
# plt.title('Three stars')

# axs.append(plt.subplot(1,5,4))
# sns.histplot(
#     data=(
#         joined
#         .loc[(joined['stars']==4) & joined['GRADE'].isin(['A','B','C']), ['stars', 'price_level','GRADE']]
#         .sort_values('GRADE'))
#     ,
#     x='stars',
#     hue='GRADE',
#     multiple='stack',
#     palette='Set2',
#     stat='percent'
# )
# plt.title('Four stars')

# axs.append(plt.subplot(1,5,5))
# sns.histplot(
#     data=(
#         joined
#         .loc[(joined['stars']==4) & joined['GRADE'].isin(['A','B','C']), ['stars', 'price_level','GRADE']]
#         .sort_values('GRADE'))
#     ,
#     x='stars',
#     hue='GRADE',
#     multiple='stack',
#     palette='Set2',
#     stat='percent'
# )
# plt.title('Five stars')

# for ax in axs[1:]:
#     ax.set(ylabel=None)
#     ax.set(yticks=[])
# for ax in axs:
#     ax.set(xlabel=None)
#     ax.set(xticks=[])
# for ax in axs[:-1]:
#     ax.get_legend().remove()
    

# plt.tight_layout()